# Chapter 8: Control Flow and State Management 실습

이 노트북은 LangGraph의 고급 제어 흐름과 상태 관리 기법을 실습합니다.

## 환경 설정

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = input("OpenAI API Key를 입력하세요: ")

## 1. Structured Output with State

In [ ]:
from typing import TypedDict, List, Dict, Optional
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END, MessagesState
from pydantic import BaseModel, Field
from datetime import datetime

# 구조화된 출력 스키마들
class TaskInfo(BaseModel):
    title: str = Field(description="작업 제목")
    priority: str = Field(description="우선순위: high/medium/low")
    estimated_hours: float = Field(description="예상 소요 시간")
    dependencies: List[str] = Field(description="의존성 목록")

class ProjectPlan(BaseModel):
    project_name: str = Field(description="프로젝트 이름")
    objectives: List[str] = Field(description="프로젝트 목표")
    tasks: List[TaskInfo] = Field(description="작업 목록")
    timeline: str = Field(description="전체 일정")
    risks: List[str] = Field(description="잠재적 위험")

# State 정의
class StructuredState(MessagesState):
    project_plan: Optional[ProjectPlan]
    validation_results: Dict
    optimization_suggestions: List[str]

# 프로젝트 계획 생성 노드
def generate_project_plan(state: StructuredState):
    """구조화된 프로젝트 계획 생성"""
    messages = state["messages"]
    query = messages[-1].content if messages else ""
    
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
    structured_llm = llm.with_structured_output(ProjectPlan)
    
    planning_prompt = f"""
    다음 요구사항에 대한 상세한 프로젝트 계획을 수립하세요:
    
    {query}
    
    체계적이고 실행 가능한 계획을 만드세요.
    """
    
    project_plan = structured_llm.invoke(planning_prompt)
    state["project_plan"] = project_plan
    
    print(f"프로젝트 계획 생성: {project_plan.project_name}")
    print(f"작업 수: {len(project_plan.tasks)}")
    
    return state

# 검증 노드
def validate_plan(state: StructuredState):
    """계획의 유효성 검증"""
    plan = state.get("project_plan")
    if not plan:
        return state
    
    validation_results = {
        "has_objectives": len(plan.objectives) > 0,
        "has_tasks": len(plan.tasks) > 0,
        "reasonable_timeline": len(plan.timeline) > 0,
        "risks_identified": len(plan.risks) > 0,
        "total_hours": sum(task.estimated_hours for task in plan.tasks),
        "high_priority_count": sum(1 for task in plan.tasks if task.priority == "high")
    }
    
    # 의존성 체크
    all_task_titles = [task.title for task in plan.tasks]
    invalid_dependencies = []
    for task in plan.tasks:
        for dep in task.dependencies:
            if dep and dep not in all_task_titles:
                invalid_dependencies.append(f"{task.title} -> {dep}")
    
    validation_results["invalid_dependencies"] = invalid_dependencies
    validation_results["is_valid"] = (
        validation_results["has_objectives"] and
        validation_results["has_tasks"] and
        len(invalid_dependencies) == 0
    )
    
    state["validation_results"] = validation_results
    
    print(f"검증 완료: {'유효' if validation_results['is_valid'] else '무효'}")
    return state

# 최적화 제안 노드
def optimize_plan(state: StructuredState):
    """계획 최적화 제안"""
    plan = state.get("project_plan")
    validation = state.get("validation_results", {})
    
    suggestions = []
    
    if validation.get("total_hours", 0) > 160:
        suggestions.append("총 작업 시간이 160시간을 초과합니다. 작업을 단계별로 나누는 것을 고려하세요.")
    
    if validation.get("high_priority_count", 0) > len(plan.tasks) * 0.5:
        suggestions.append("고우선순위 작업이 너무 많습니다. 우선순위를 재조정하세요.")
    
    if len(plan.risks) < 3:
        suggestions.append("더 많은 잠재적 위험을 식별하여 리스크 관리를 강화하세요.")
    
    # 병렬 처리 가능 작업 찾기
    parallel_tasks = []
    for task in plan.tasks:
        if not task.dependencies:
            parallel_tasks.append(task.title)
    
    if len(parallel_tasks) > 1:
        suggestions.append(f"병렬 처리 가능: {', '.join(parallel_tasks)}")
    
    state["optimization_suggestions"] = suggestions
    
    return state

# 구조화된 출력 그래프 생성
def create_structured_output_graph():
    workflow = StateGraph(StructuredState)
    
    workflow.add_node("generate", generate_project_plan)
    workflow.add_node("validate", validate_plan)
    workflow.add_node("optimize", optimize_plan)
    
    workflow.set_entry_point("generate")
    workflow.add_edge("generate", "validate")
    workflow.add_edge("validate", "optimize")
    workflow.add_edge("optimize", END)
    
    return workflow.compile()

# 실행
structured_graph = create_structured_output_graph()

# 테스트
project_requests = [
    "모바일 쇼핑 앱 개발 프로젝트",
    "회사 웹사이트 리뉴얼 프로젝트"
]

for request in project_requests:
    print(f"\n{'='*80}")
    print(f"요청: {request}\n")
    
    result = structured_graph.invoke({
        "messages": [HumanMessage(content=request)]
    })
    
    plan = result["project_plan"]
    print(f"\n프로젝트: {plan.project_name}")
    print(f"목표: {', '.join(plan.objectives[:2])}...")
    print(f"\n작업 목록:")
    for task in plan.tasks[:3]:
        print(f"  - {task.title} ({task.priority}, {task.estimated_hours}h)")
    
    print(f"\n최적화 제안:")
    for suggestion in result["optimization_suggestions"]:
        print(f"  • {suggestion}")

## 2. Streaming Output

In [ ]:
import asyncio
from typing import AsyncIterator

# Streaming State
class StreamingState(MessagesState):
    current_chunk: str
    accumulated_response: str
    metadata: Dict

# 스트리밍 노드
async def stream_response(state: StreamingState):
    """스트리밍 응답 생성"""
    messages = state["messages"]
    
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, streaming=True)
    
    prompt = """
    사용자 질문에 대해 상세하고 도움이 되는 답변을 제공하세요.
    단계별로 설명하고 예시를 포함하세요.
    """
    
    full_response = ""
    async for chunk in llm.astream([{"role": "system", "content": prompt}] + messages):
        if chunk.content:
            full_response += chunk.content
            yield chunk.content
    
    state["accumulated_response"] = full_response
    return state

# 스트리밍 파이프라인
class StreamingPipeline:
    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
    
    async def process_with_progress(self, query: str):
        """진행 상황을 표시하며 처리"""
        stages = [
            ("분석 중", self._analyze),
            ("연구 중", self._research),
            ("작성 중", self._write),
            ("검토 중", self._review)
        ]
        
        results = {}
        for stage_name, stage_func in stages:
            print(f"\n[{stage_name}]", end="")
            result = await stage_func(query, results)
            results[stage_name] = result
            
            # 진행 표시
            for _ in range(3):
                print(".", end="", flush=True)
                await asyncio.sleep(0.5)
            print(" 완료!")
        
        return results
    
    async def _analyze(self, query: str, context: dict) -> str:
        response = await self.llm.ainvoke(f"분석: {query}")
        return response.content[:100]
    
    async def _research(self, query: str, context: dict) -> str:
        response = await self.llm.ainvoke(f"연구: {query}")
        return response.content[:100]
    
    async def _write(self, query: str, context: dict) -> str:
        analysis = context.get("분석 중", "")
        response = await self.llm.ainvoke(f"작성: {query}\n분석: {analysis}")
        return response.content[:200]
    
    async def _review(self, query: str, context: dict) -> str:
        writing = context.get("작성 중", "")
        response = await self.llm.ainvoke(f"검토: {writing[:50]}")
        return response.content[:100]

# 실시간 스트리밍 시뮬레이션
async def simulate_streaming():
    """실시간 스트리밍 시뮬레이션"""
    queries = [
        "인공지능의 미래는?",
        "효과적인 학습 방법은?"
    ]
    
    for query in queries:
        print(f"\n{'='*60}")
        print(f"질문: {query}")
        print("="*60)
        
        # 스트리밍 응답
        print("\n스트리밍 응답:")
        llm = ChatOpenAI(model="gpt-4o-mini", streaming=True)
        
        full_response = ""
        async for chunk in llm.astream(query):
            if chunk.content:
                print(chunk.content, end="", flush=True)
                full_response += chunk.content
                await asyncio.sleep(0.01)  # 시각적 효과
        
        print("\n\n처리 완료!")

# 파이프라인 실행
async def run_pipeline():
    pipeline = StreamingPipeline()
    
    query = "지속 가능한 발전을 위한 전략"
    print(f"\n질문: {query}")
    
    results = await pipeline.process_with_progress(query)
    
    print("\n최종 결과:")
    for stage, result in results.items():
        print(f"\n[{stage}]")
        print(result[:100] + "...")

# 실행
print("스트리밍 데모 시작...")
await simulate_streaming()
await run_pipeline()

## 3. Interrupt and Human-in-the-Loop

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from typing import Literal

# Human-in-the-Loop State
class HILState(MessagesState):
    draft_response: str
    human_feedback: Optional[str]
    approval_status: Optional[str]
    revision_count: int

# 초안 생성 노드
def generate_draft_response(state: HILState):
    """초안 응답 생성"""
    messages = state["messages"]
    
    llm = ChatOpenAI(model="gpt-4o-mini")
    response = llm.invoke(messages)
    
    state["draft_response"] = response.content
    state["revision_count"] = state.get("revision_count", 0)
    
    print(f"\n초안 생성 완료 (수정 {state['revision_count']}회)")
    print(f"초안: {response.content[:200]}...")
    
    return state

# 인간 검토 시뮬레이션
def simulate_human_review(draft: str) -> tuple[str, str]:
    """인간 검토 시뮬레이션 (실제로는 사용자 입력)"""
    print("\n=== 인간 검토 필요 ===")
    print(f"초안: {draft[:300]}...")
    print("\n옵션:")
    print("1. 승인 (approve)")
    print("2. 수정 요청 (revise)")
    print("3. 거부 (reject)")
    
    # 시뮬레이션: 랜덤 선택
    import random
    choice = random.choice(["approve", "revise", "revise", "approve"])
    
    feedback = ""
    if choice == "revise":
        feedback = "더 구체적인 예시를 추가하고 전문 용어를 줄여주세요."
    
    print(f"\n선택: {choice}")
    if feedback:
        print(f"피드백: {feedback}")
    
    return choice, feedback

# 인간 검토 노드
def human_review(state: HILState):
    """인간 검토 단계"""
    draft = state["draft_response"]
    
    # 실제로는 여기서 중단하고 사용자 입력을 기다림
    approval, feedback = simulate_human_review(draft)
    
    state["approval_status"] = approval
    state["human_feedback"] = feedback
    
    return state

# 수정 노드
def revise_based_on_feedback(state: HILState):
    """피드백 기반 수정"""
    draft = state["draft_response"]
    feedback = state["human_feedback"]
    
    llm = ChatOpenAI(model="gpt-4o-mini")
    
    revision_prompt = f"""
    원본 응답:
    {draft}
    
    피드백:
    {feedback}
    
    피드백을 반영하여 응답을 수정하세요.
    """
    
    revised = llm.invoke(revision_prompt)
    state["draft_response"] = revised.content
    state["revision_count"] += 1
    
    print(f"\n수정 완료 (총 {state['revision_count']}회 수정)")
    
    return state

# 라우팅 함수
def route_after_review(state: HILState) -> Literal["revise", "finalize", "reject"]:
    """검토 후 라우팅"""
    status = state.get("approval_status")
    
    if status == "approve":
        return "finalize"
    elif status == "revise":
        if state.get("revision_count", 0) >= 3:
            print("최대 수정 횟수 도달")
            return "finalize"
        return "revise"
    else:
        return "reject"

# 최종화 노드
def finalize_response(state: HILState):
    """응답 최종화"""
    print("\n✅ 응답 최종 승인됨")
    return {"messages": [AIMessage(content=state["draft_response"])]}

# 거부 노드
def handle_rejection(state: HILState):
    """거부 처리"""
    print("\n❌ 응답 거부됨")
    return {"messages": [AIMessage(content="죄송합니다. 적절한 응답을 생성할 수 없습니다.")]}

# Human-in-the-Loop 그래프 생성
def create_hil_graph():
    workflow = StateGraph(HILState)
    
    # 노드 추가
    workflow.add_node("generate", generate_draft_response)
    workflow.add_node("review", human_review)
    workflow.add_node("revise", revise_based_on_feedback)
    workflow.add_node("finalize", finalize_response)
    workflow.add_node("reject", handle_rejection)
    
    # 플로우 정의
    workflow.set_entry_point("generate")
    workflow.add_edge("generate", "review")
    workflow.add_edge("revise", "review")
    
    # 조건부 라우팅
    workflow.add_conditional_edges(
        "review",
        route_after_review,
        {
            "revise": "revise",
            "finalize": "finalize",
            "reject": "reject"
        }
    )
    
    workflow.add_edge("finalize", END)
    workflow.add_edge("reject", END)
    
    # 체크포인터 추가
    memory = MemorySaver()
    return workflow.compile(checkpointer=memory)

# 실행
hil_graph = create_hil_graph()

# 테스트
queries = [
    "복잡한 프로젝트를 관리하는 방법",
    "AI 윤리에 대한 고찰"
]

for query in queries:
    print(f"\n{'='*80}")
    print(f"질문: {query}")
    
    config = {"configurable": {"thread_id": f"session_{query[:10]}"}}
    
    result = hil_graph.invoke(
        {"messages": [HumanMessage(content=query)]},
        config
    )
    
    print(f"\n최종 응답:")
    print(result["messages"][-1].content[:500])

## 4. State Editing and Forking

In [ ]:
import copy
from typing import Any

# 버전 관리 State
class VersionedState(MessagesState):
    version: int
    branch_name: str
    history: List[Dict[str, Any]]
    current_data: Dict
    forks: List[str]

# State 버전 관리자
class StateVersionManager:
    def __init__(self):
        self.branches = {}
        self.current_branch = "main"
    
    def save_state(self, state: Dict, branch: str = None) -> str:
        """현재 상태 저장"""
        branch = branch or self.current_branch
        
        if branch not in self.branches:
            self.branches[branch] = []
        
        version = len(self.branches[branch])
        snapshot = {
            "version": version,
            "timestamp": datetime.now().isoformat(),
            "state": copy.deepcopy(state)
        }
        
        self.branches[branch].append(snapshot)
        return f"{branch}:v{version}"
    
    def fork_state(self, from_branch: str, to_branch: str, version: int = -1) -> Dict:
        """상태 포크"""
        if from_branch not in self.branches:
            raise ValueError(f"Branch {from_branch} not found")
        
        source_state = self.branches[from_branch][version]["state"]
        forked_state = copy.deepcopy(source_state)
        
        # 새 브랜치 생성
        self.branches[to_branch] = [{
            "version": 0,
            "timestamp": datetime.now().isoformat(),
            "state": forked_state,
            "forked_from": f"{from_branch}:v{version}"
        }]
        
        return forked_state
    
    def rollback(self, branch: str, version: int) -> Dict:
        """특정 버전으로 롤백"""
        if branch not in self.branches:
            raise ValueError(f"Branch {branch} not found")
        
        if version >= len(self.branches[branch]):
            raise ValueError(f"Version {version} not found")
        
        return self.branches[branch][version]["state"]
    
    def merge_states(self, branch1: str, branch2: str, strategy: str = "latest") -> Dict:
        """두 브랜치 병합"""
        state1 = self.branches[branch1][-1]["state"]
        state2 = self.branches[branch2][-1]["state"]
        
        if strategy == "latest":
            # 가장 최근 것 선택
            time1 = self.branches[branch1][-1]["timestamp"]
            time2 = self.branches[branch2][-1]["timestamp"]
            return state1 if time1 > time2 else state2
        elif strategy == "combine":
            # 두 상태 결합
            merged = copy.deepcopy(state1)
            for key, value in state2.items():
                if key not in merged:
                    merged[key] = value
                elif isinstance(value, list) and isinstance(merged[key], list):
                    merged[key].extend(value)
            return merged
        
        return state1

# State 편집 노드들
def edit_state_node(state: VersionedState, manager: StateVersionManager):
    """상태 편집"""
    # 현재 상태 저장
    version_id = manager.save_state(state, state.get("branch_name", "main"))
    
    # 상태 수정
    state["version"] = state.get("version", 0) + 1
    state["current_data"]["edited"] = True
    state["current_data"]["edit_time"] = datetime.now().isoformat()
    
    # 히스토리 추가
    if "history" not in state:
        state["history"] = []
    state["history"].append({
        "action": "edit",
        "version_id": version_id,
        "timestamp": datetime.now().isoformat()
    })
    
    print(f"상태 편집 완료: {version_id}")
    return state

def fork_workflow(state: VersionedState, manager: StateVersionManager, fork_name: str):
    """워크플로우 포크"""
    current_branch = state.get("branch_name", "main")
    
    # 포크 생성
    forked_state = manager.fork_state(current_branch, fork_name)
    
    # 포크 정보 업데이트
    forked_state["branch_name"] = fork_name
    forked_state["version"] = 0
    
    if "forks" not in state:
        state["forks"] = []
    state["forks"].append(fork_name)
    
    print(f"포크 생성: {current_branch} -> {fork_name}")
    return forked_state

# 실험적 워크플로우
class ExperimentalWorkflow:
    def __init__(self):
        self.manager = StateVersionManager()
        self.experiments = {}
    
    def run_experiment(self, name: str, initial_state: Dict):
        """실험 실행"""
        print(f"\n실험 시작: {name}")
        
        # 초기 상태 저장
        self.manager.save_state(initial_state, name)
        
        # 실험 단계들
        stages = [
            ("preprocessing", self._preprocess),
            ("analysis", self._analyze),
            ("optimization", self._optimize)
        ]
        
        current_state = initial_state
        for stage_name, stage_func in stages:
            print(f"  단계: {stage_name}")
            
            # 각 단계 전 상태 저장
            self.manager.save_state(current_state, name)
            
            # 단계 실행
            current_state = stage_func(current_state)
            
            # 실패 시 롤백 가능
            if current_state.get("error"):
                print(f"    오류 발생! 이전 상태로 롤백")
                current_state = self.manager.rollback(name, -2)
                break
        
        self.experiments[name] = current_state
        return current_state
    
    def _preprocess(self, state: Dict) -> Dict:
        state["preprocessed"] = True
        state["data_size"] = 1000
        return state
    
    def _analyze(self, state: Dict) -> Dict:
        if state.get("data_size", 0) > 500:
            state["analysis_result"] = "large_dataset"
        else:
            state["analysis_result"] = "small_dataset"
        return state
    
    def _optimize(self, state: Dict) -> Dict:
        if state.get("analysis_result") == "large_dataset":
            state["optimization"] = "parallel_processing"
        else:
            state["optimization"] = "single_thread"
        return state
    
    def compare_experiments(self, exp1: str, exp2: str):
        """실험 결과 비교"""
        result1 = self.experiments.get(exp1)
        result2 = self.experiments.get(exp2)
        
        if not result1 or not result2:
            return "실험을 찾을 수 없습니다."
        
        print(f"\n실험 비교: {exp1} vs {exp2}")
        print(f"{exp1}: {result1.get('optimization', 'N/A')}")
        print(f"{exp2}: {result2.get('optimization', 'N/A')}")

# 실행
print("State 버전 관리 데모")

# 버전 관리자 테스트
manager = StateVersionManager()

# 초기 상태
initial_state = {
    "version": 0,
    "branch_name": "main",
    "current_data": {"value": 100},
    "messages": []
}

# 상태 편집
state1 = edit_state_node(initial_state.copy(), manager)
print(f"버전 1: {state1['version']}")

# 포크 생성
forked_state = fork_workflow(state1.copy(), manager, "experiment-1")
print(f"포크된 브랜치: {forked_state['branch_name']}")

# 실험 워크플로우
print("\n실험 워크플로우 테스트")
workflow = ExperimentalWorkflow()

# 여러 실험 실행
exp1_state = {"experiment": "A", "parameters": {"learning_rate": 0.01}}
exp2_state = {"experiment": "B", "parameters": {"learning_rate": 0.001}}

result1 = workflow.run_experiment("exp_A", exp1_state)
result2 = workflow.run_experiment("exp_B", exp2_state)

# 실험 비교
workflow.compare_experiments("exp_A", "exp_B")

## 5. Advanced Control Flow Patterns

In [ ]:
from typing import Union
import time

# 복잡한 제어 흐름 State
class ComplexFlowState(TypedDict):
    task: str
    retry_count: int
    max_retries: int
    timeout: float
    parallel_tasks: List[Dict]
    conditional_branches: Dict
    loop_counter: int
    exit_condition: bool
    results: List[Any]

# 재시도 로직
def retry_on_failure(func):
    """재시도 데코레이터"""
    def wrapper(state: ComplexFlowState):
        max_retries = state.get("max_retries", 3)
        retry_count = state.get("retry_count", 0)
        
        while retry_count < max_retries:
            try:
                result = func(state)
                state["retry_count"] = 0
                return result
            except Exception as e:
                retry_count += 1
                state["retry_count"] = retry_count
                print(f"시도 {retry_count}/{max_retries} 실패: {e}")
                
                if retry_count >= max_retries:
                    print("최대 재시도 횟수 초과")
                    state["error"] = str(e)
                    return state
                
                time.sleep(2 ** retry_count)  # 지수 백오프
        
        return state
    return wrapper

# 타임아웃 처리
def with_timeout(timeout_seconds: float):
    """타임아웃 데코레이터"""
    def decorator(func):
        def wrapper(state: ComplexFlowState):
            import signal
            
            def timeout_handler(signum, frame):
                raise TimeoutError(f"작업이 {timeout_seconds}초를 초과했습니다")
            
            # 타임아웃 설정 (Unix 시스템에서만 작동)
            try:
                signal.signal(signal.SIGALRM, timeout_handler)
                signal.alarm(int(timeout_seconds))
                result = func(state)
                signal.alarm(0)
                return result
            except:
                # Windows 등에서는 간단한 시뮬레이션
                import threading
                result = [None]
                
                def run():
                    result[0] = func(state)
                
                thread = threading.Thread(target=run)
                thread.start()
                thread.join(timeout_seconds)
                
                if thread.is_alive():
                    print(f"타임아웃: {timeout_seconds}초 초과")
                    state["timeout_occurred"] = True
                    return state
                
                return result[0]
        return wrapper
    return decorator

# 병렬 처리 노드
async def parallel_execution(state: ComplexFlowState):
    """병렬 작업 실행"""
    parallel_tasks = state.get("parallel_tasks", [])
    
    async def execute_task(task: Dict):
        llm = ChatOpenAI(model="gpt-4o-mini")
        response = await llm.ainvoke(task["prompt"])
        return {
            "task_id": task["id"],
            "result": response.content[:100]
        }
    
    # 모든 작업 동시 실행
    import asyncio
    results = await asyncio.gather(*[execute_task(task) for task in parallel_tasks])
    
    state["results"] = results
    print(f"병렬 처리 완료: {len(results)}개 작업")
    
    return state

# 조건부 분기
def conditional_branching(state: ComplexFlowState) -> str:
    """조건에 따른 분기"""
    task = state.get("task", "")
    
    conditions = {
        "simple": lambda: len(task) < 50,
        "complex": lambda: "analyze" in task.lower() or "research" in task.lower(),
        "urgent": lambda: "urgent" in task.lower() or "asap" in task.lower()
    }
    
    for branch_name, condition in conditions.items():
        if condition():
            print(f"분기 선택: {branch_name}")
            return branch_name
    
    return "default"

# 반복 처리
def loop_until_condition(state: ComplexFlowState):
    """조건을 만족할 때까지 반복"""
    loop_counter = state.get("loop_counter", 0)
    max_loops = 5
    
    while loop_counter < max_loops and not state.get("exit_condition"):
        print(f"반복 {loop_counter + 1}/{max_loops}")
        
        # 작업 수행
        llm = ChatOpenAI(model="gpt-4o-mini")
        response = llm.invoke(f"반복 {loop_counter}: {state.get('task', '')}")
        
        # 종료 조건 체크
        if "complete" in response.content.lower():
            state["exit_condition"] = True
        
        loop_counter += 1
        state["loop_counter"] = loop_counter
    
    if state.get("exit_condition"):
        print("조건 만족 - 반복 종료")
    else:
        print("최대 반복 횟수 도달")
    
    return state

# 복잡한 제어 흐름 그래프
def create_complex_flow_graph():
    workflow = StateGraph(ComplexFlowState)
    
    # 재시도 가능한 노드
    @retry_on_failure
    def risky_operation(state):
        import random
        if random.random() < 0.5:
            raise Exception("임의 오류 발생")
        state["risky_result"] = "성공"
        return state
    
    # 타임아웃이 있는 노드
    @with_timeout(3.0)
    def slow_operation(state):
        time.sleep(1)  # 시뮬레이션
        state["slow_result"] = "완료"
        return state
    
    # 노드 추가
    workflow.add_node("risky", risky_operation)
    workflow.add_node("slow", slow_operation)
    workflow.add_node("loop", loop_until_condition)
    
    # 분기별 처리 노드
    def simple_handler(state):
        state["result"] = "간단한 작업 완료"
        return state
    
    def complex_handler(state):
        state["result"] = "복잡한 분석 완료"
        return state
    
    def urgent_handler(state):
        state["result"] = "긴급 처리 완료"
        return state
    
    def default_handler(state):
        state["result"] = "기본 처리 완료"
        return state
    
    workflow.add_node("simple", simple_handler)
    workflow.add_node("complex", complex_handler)
    workflow.add_node("urgent", urgent_handler)
    workflow.add_node("default", default_handler)
    
    # 플로우 정의
    workflow.set_entry_point("risky")
    workflow.add_edge("risky", "slow")
    workflow.add_edge("slow", "loop")
    
    # 조건부 분기
    workflow.add_conditional_edges(
        "loop",
        conditional_branching,
        {
            "simple": "simple",
            "complex": "complex",
            "urgent": "urgent",
            "default": "default"
        }
    )
    
    # 모든 분기를 END로
    for node in ["simple", "complex", "urgent", "default"]:
        workflow.add_edge(node, END)
    
    return workflow.compile()

# 실행
print("복잡한 제어 흐름 테스트\n")

complex_graph = create_complex_flow_graph()

# 다양한 시나리오 테스트
test_scenarios = [
    {"task": "간단한 작업", "max_retries": 3},
    {"task": "복잡한 데이터 analyze 작업", "max_retries": 2},
    {"task": "urgent: 긴급 처리 필요", "max_retries": 1}
]

for scenario in test_scenarios:
    print(f"\n{'='*60}")
    print(f"시나리오: {scenario['task']}")
    
    result = complex_graph.invoke(scenario)
    
    print(f"\n최종 결과: {result.get('result', 'N/A')}")
    if result.get('risky_result'):
        print(f"위험 작업: {result['risky_result']}")
    if result.get('slow_result'):
        print(f"느린 작업: {result['slow_result']}")
    print(f"반복 횟수: {result.get('loop_counter', 0)}")

## 실습 과제

1. 동적 라우팅과 상태 관리를 결합한 적응형 워크플로우
2. 실시간 모니터링과 자동 복구 기능이 있는 안정적인 시스템
3. 다중 사용자 협업을 지원하는 버전 관리 시스템

In [ ]:
# 여기에 실습 코드를 작성하세요
